
# Práctica 4: Metaheurísticas basadas en poblaciones - Sistemas de Colonias de Hormigas

<center><h3>
    Uxía Carro Tacón
</h3></center>


## Instrucciones

Esto es **Jupyter Notebook**, un documento que integra código Python en un archivo Markdown.
Esto nos permite ir ejecutando celdas de código poco a poco, así como generar automáticamente un informe bien formateado de la práctica.

Puedes añadir una celda con el botón **"Insert"** de la barra de herramentas, y cambiar su tipo con **"Cell > Cell Type"**

Para ejecutar una celda de código, la seleccionaremos y pulsaremos el botón **"▶ Run"** de la barra de herramentas.
Para pasar el documento a HTML, seleccionaremos **"File > Download as > HTML (.html)"**

Sigue este guión hasta el final. Ejecuta el código proporcionado paso a paso comprendiendo lo que estás haciendo y reflexionando sobre los resultados. Habrá preguntas intercaladas a lo largo del guión, responde a todas ellas en la sección reservada para ese fin: **"Respuestas a los cuestionarios"**. Por favor, no modifiques ninguna linea de código excepto cuando se te pida explícitamente.

No olvides insertar tu **nombre y apellidos** en la celda superior.

IMPORTANTE: Escribe el código de tu o tus soluciones/respuestas en las celdas que se indican para ello. Además, a lo largo de la práctica se plantearán varias preguntas que debéis responder en la parte inferior del documento, incluyendo las celdas que veáis necesarias (si hacéis referencia a partes concretas de vuestro código, etc) para reponder a ellas.

## Entrega de la práctica

La fecha límite de entrega será la indicada en el Campus Virtual. La entrega consistirá de un único archivo comprimido con nombre `APELIDOS_NOME_Colonias.zip` que contenga los seguientes ficheros:

 * `APELIDOS_NOME_Colonias.html`: Archivo HTML fruto de la exportación del presente Notebook, con las preguntas respondidas al final del documento.
 * `APELIDOS_NOME_Colonias.ipynb`: Archivo fuente Jupyter Notebook.
 * Archivo de datos de los problema utilizados en la resolución.

## El Problema del Viajante de Comercio (VC) con Sistemas de Colonias de Hormigas

El objetivo de esta práctica es modelar e implementar un agente inteligente que sea capaz de resolver
el problema del VC mediante la metaheurística (MH) de algoritmos para sistemas de colonias de
hormigas (ACS, *Ant Colony Systems*, del inglés).

Para ello, realizarás una implementación del algoritmo básico visto en la clase expositiva y valorarás algunos de los parámetros de diseño del algoritmo en relación a la calidad de las soluciones alcanzadas.

Recuerda que debes importar el módulo Python que acompaña esta práctica, que como ocurría en prácticas anteriores implementa la carga de datos y el cálculo de distancias utilizando una matriz de adyacencia de acuerdo a la definición del problema ya vista.

In [2]:
from helpers_mod_aco import *

g1=Localizaciones(filename='./data/grafo8cidades.txt')

# distancia entre ciudad 0 y 1 son unos 55 km
print (g1.distancia(0,1))

55.88273580792048



## P4.1: Implementación de Sistemas de Colonias de Hormigas (SCH)


Implementa el algoritmo de sistemas de colonia de hormigas que materializa la MH vista en la sesión expositiva y revisa las notas particulares para el problema de VC. Para facilitar vuestra labor, se os proporciona aquí una descripción algorítmica de referencia.


&nbsp;&nbsp;&nbsp;&nbsp;1. Establecer parámetros: $G, m, q_{0}, \tau_{0}, \rho, \xi, iter_{min}, iter_{max}, iter_{sinmejoras}, ratio_{mejora}$

&nbsp;&nbsp;&nbsp;&nbsp;2. Inicializar pistas de feromonas: $\forall (i, j)  \tau_{ij} = \tau_{0}$

&nbsp;&nbsp;&nbsp;&nbsp;3. HACER
        
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;3.1. PARA cada hormiga $k$ en colonia m HACER
    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;3.1.1.construir una solución para $k$ siguiendo la regla de decisión a cada paso:
$$ j=\begin{cases}argmax_{l\in N_i^k} \{ [\tau_{il}]^\alpha [\eta_{il}]^\beta \}, \text{if } q \leq q_0; \\ J, \text{en otro caso;}\end{cases}
$$,
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;donde $J$ es la exploración dirigida por: 

$$ p_{ij}^{k} = \frac{[\tau_{ij}]^\alpha [\eta_{ij}]^\beta}{\sum_{l\in N_i^k} [\tau_{il}]^\alpha [\eta_{il}]^\beta}, \text{if } j\in N_i^k 
$$    

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;3.1.2. (opcional) Aplicar actualización de feromona local/online con la regla:
$$ \tau_{ij} \leftarrow (1 - \xi) \tau_{ij} + \xi \tau_{0}$$
     
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;3.2. FIN PARA
   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;3.3. Evalúa soluciones en la colonia m y registrar la mejor solución hasta el momento $T^{bs}$
    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;3.4. Aplicar actualización de feromona global/offline con la regla: $$\tau_{ij} \leftarrow (1-\rho) \tau_{ij} + \rho \Delta\tau_{ij}^{bs}, \forall(i, j) \in T^{bs}$$
    

&nbsp;&nbsp;&nbsp;&nbsp;4. HASTA cumplir la condición de parada




La implementación debe permitir la inicialización de los parámetros y estructuras de información asociadas como sigue:
- G es la estructura de grafo que permite calcular distancias entre ciudades sobre una instancia del problema dada (e.g., fichero de localizaciones).
- Probabilidad de explotación $q_{0}=0.5$.
- Inicialización de la información heurística: $\eta_{ij}=1/d_{ij}, \forall (i, j)$, donde $d_{ij}$ es la distancia directa entre las ciudades *i* y *j*.
- Inicialización de las pistas de feromonas (depósito inicial): $\tau_{ij}= \tau_{0}= 1 /(n C^{NN}) \forall (i, j)$ donde n es el número de ciudades y $C^{NN}$ es el coste del tour generado siguiendo una estrategia voraz del vecino más próximo (NN, *Nearest Neighbour*).
- Parámetros de influencia de las pistas de feromonas ($\alpha$) e información heurística ($\beta$) en la regla de decisión establecidos a 1 y 3 respectivamente 
- Ratio de evaporación de feromona: deberás probar con diferentes magnitudes (p.ej., $\rho=0.01$, $\rho=0.1$ y $\rho=0.5$)
- Actualización de feromona global con depósito elitista basado en la mejor solución (bs, *best solution*):
    $\tau_{ij} \leftarrow (1-\rho) \tau_{ij} + \rho \Delta\tau_{ij}^{bs}, \forall (i, j) \in T^{bs}, donde \Delta\tau_{ij}^{bs}=1/C^{bs}$
- Actualización de feromona local siguiendo la regla:
    $\tau_{ij} \leftarrow (1- \xi) \tau_{ij} + \xi \tau_{0}, con  \xi=0.1$
- Condición de parada que de manera combinada tenga en cuenta $iter_{min}$, $iter_{max}$, $iter_{sinmejoras}$, $ratio_{mejora}$
- La implementación debe ser completamente parametrizable, de forma que todos los componentes susceptibles de ser ajustados mediante parámetros puedan ser establecidos en cada ejecución/problema, y que puedas modificar la inicialización heurística y de pistas de feromona, puedas fácilmente reemplazar las reglas de actualización de feromonas, activar/desactivar la actualización local de actualización de feromona, etc.

Prueba tu implementación tomando instancias de problemas que has utilizado en prácticas anteriores. Lanza varias ejecuciones para verificar que puede resolver el problema con los problemas de las 8 ciudades gallegas (`data/grafo8cidades.txt`) y de las 120 ciudades estadounidenses (`data/US120.txt`).


In [14]:
pip install numpy

     |████████████████████████████████| 15.7MB 7.3MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Escribe aquí tu código para la(s) función(es) que implementa tu algoritmo para el sistema de colonias de hormigas
# Crea tantas celdas como consideres oportuno para escribir tu código
# Documenta siempre tu código con comentarios como este

import random
import time
import numpy as np

def greedy(g):
    #número de ciudades
    n=g.nciudades
    #lista de ciudades visitadas del tamaño del número de ciudades
    visitadas=[False]*n
    #lista de la solucion
    solucion=[]
    #se empieza por la ciudad 0, pero sin añadirla
    ciudad=0

    for i in range(n-1):
        visitadas[ciudad]=True
        #se añade la ciudad a la solucion
        solucion.append(ciudad)
        #se busca la ciudad más cercana a la ciudad actual
        ciudad_cercana=-1
        #ciudad a minimizar empieza siendo infinito
        ciudad_minimizar=float('inf')
        for j in range(n):
            #si la ciudad no ha sido visitada 
            if (not visitadas[j]):
                #se calcula la distancia entre la ciudad actual y la ciudad j
                distancia=g.distancia(ciudad,j)
                #si la distancia es menor que la distancia minima se actualiza la distancia minima y 
                #la ciudad más cercana
                if (distancia<ciudad_minimizar):
                    ciudad_minimizar=distancia
                    ciudad_cercana=j
        #se actualiza la ciudad actual
        ciudad=ciudad_cercana
    #se añade la ciudad 0 al final de la solucion
    solucion.append(ciudad)
    #se devuleve la solucion empezando por la ciudad 1 para eliminar la ciudad 0
    return solucion[1:]

def coste(g, solucionI):
    solucion = [0]+solucionI+[0]
    suma_coste = 0
    #se van sumando las distancias entre las ciudades de la solucion
    for i in range(0, len(solucion)-1):
        suma_coste += g.distancia(solucion[i], solucion[i+1])
    return suma_coste

# funcion de generacion de vencindario
def genera_vecindario(solucion, g):
    #aqui el vecindario son todos menos los ya visitados
    #número de ciudades
    n=g.nciudades
    #lista de ciudades visitadas del tamaño del número de ciudades
    visitadas=[False]*n
    #se marcan como visitadas las ciudades de la solucion
    for i in solucion:
        visitadas[i]=True
    #lista de ciudades no visitadas
    no_visitadas=[]
    #se añaden a la lista las ciudades no visitadas
    for i in range(n):
        if (not visitadas[i]):
            no_visitadas.append(i)
    #se devuelve el vecindario
    return no_visitadas

def parada(itermax, itermin, iteracion, itersinmejoras, ratiomejora):
    if (iteracion==itermax):
        return True
    if(itersinmejoras==100 and iteracion > itermin):
        return True
    if(ratiomejora<0.1 and iteracion > itermin):
        return True
    return False

def inicializar_feromonas(g):
    tau0 = 1/(g.nciudades*coste(g, greedy(g)))
    tau = [[tau0]*g.nciudades for i in range(g.nciudades)]
    return tau, tau0



In [37]:
def hormigas(p=0.1,q0=0.5,eps=0.1, alfa=1, beta=3, itermax=150, itermin=10, actualizacionLocal = False, m=10, filename='./data/grafo8cidades.txt'):
    g = Localizaciones(filename=filename)
    mejor_solucion=[]
    mejor_coste=float('inf')
    itersinmejoras=0
    ratiomejora=0
    
    #matriz de la información heurística
    n=[[0]*g.nciudades for i in range(g.nciudades)]
    for i in range(g.nciudades):
        for j in range(g.nciudades):
            if (i==j):
                n[i][j] = 0
            else:
                n[i][j]=1/g.distancia(i,j)
    #matriz de pista de feromonas
    tau, tau0 = inicializar_feromonas(g)
    iteracion=1
    random.seed(time.time())

    while(not parada(itermax, itermin, iteracion, itersinmejoras, ratiomejora)):
        solucionesColonia=[]
        #para cada hormiga en la colonia
        for k in range(m):
            prob = random.random()
            sol=[0]
            #construimos una solucion para la hormiga paso a paso
            for p in range(g.nciudades-1):
                vecindario = genera_vecindario(sol, g)
                if (prob <= q0):
                    #desde la ultima ciudad visitada
                    i = sol[-1]
                    atracciones={}
                    for j in vecindario:
                        atraccion = (tau[i][j]**alfa)*(n[i][j]**beta)
                        atracciones[j] = atraccion
                    indice_mayor_atraccion = max(atracciones, key=atracciones.get)
                    sol.append(indice_mayor_atraccion)#la ciudad correspondiente a esta mayor atraccion
                else:
                    #si solo queda una ciudad sin visitar se añade siempre
                    if (len(vecindario)==1):
                        sol.append(vecindario[0])
                        break
                    #se calcula la suma de las atracciones de las ciudades del vecindario desde la ciudad actual
                    suma_atracciones=0
                    probabilidades={}
                    i=sol[-1]
                    for j in vecindario:
                        suma_atracciones+=(tau[i][j]**alfa)*(n[i][j]**beta)
                        #calculamos la probabilidad de cada ciudad del vecindario
                    for j in vecindario:
                        if (suma_atracciones!=0):
                            probabilidades[j] = ((tau[i][j]**alfa)*(n[i][j]**beta))/suma_atracciones
                            #probabilidades[(tau[i][j]**alfa)*(n[i][j]**beta)/suma_atracciones] = j
                    
                    #se calculan las probabilidades sobre 1
                    probtotal=sum(probabilidades.values())
                    newprobabilidades={}
                    for clave, valor in probabilidades.items():
                        newprobabilidades[clave] = (valor/probtotal)
                    
                    #se calculan las probabilidades sobre 1
                    #probtotal=sum(probabilidades.keys())
                    #newprobabilidades={}
                    #for j in probabilidades:
                    #    newprobabilidades[j/probtotal] = probabilidades.get(j)
                        
                    prob_acumulada=0
                    #se calcula la siguiente ciudad elegida
                    #se calcula un numero aleatorio entre 0 y 1
                    moneda=random.random()
                    for ciudad, prob in newprobabilidades.items():
                        prob_acumulada+=prob
                        if (moneda<=prob_acumulada):
                            sol.append(ciudad)
                            fin=1
                            break                        
            #cuando la hormiga ya tiene su solución construida
            #la añadimos a las soluciones de esta colonia (sin el 0 inicial)
            solucionesColonia.append(sol[1:])
        
            #ACTUALIZACION LOCAL DE LAS PISTAS DE FEROMONAS
            if (actualizacionLocal):
                for i in range(len(sol)-1):
                        tau[sol[i]][sol[i+1]]=(1-p)*tau[sol[i]][sol[i+1]]+p*tau0
        #cuando terminan todas las hormigas de la colonia
        #selecciono la mejor solucion de la colonia
        mejora=0
        mejor_coste_act=mejor_coste
        for ss in solucionesColonia:
            coste_solucion=coste(g, ss)
            if (coste_solucion<mejor_coste and coste_solucion>0):
                #print('ini', mejor_coste)
                #print('mejorada', coste_solucion)
                #significa q mejora
                ratio_mejora = coste_solucion/mejor_coste_act
                #print(ratio_mejora)
                mejor_coste=coste_solucion
                mejor_solucion=ss
                mejora=1
        if(mejora==1): 
            itersinmejoras=0
        else: 
            itersinmejoras+=1
        #actualizar la pista de feromonas global
        for i in range(len(mejor_solucion)-1):
            tau[mejor_solucion[i]][mejor_solucion[i+1]] = (1-eps)*tau[mejor_solucion[i]][mejor_solucion[i+1]]+eps*(1/coste(g, mejor_solucion))
            #tau[mejor_solucion[i+1]][mejor_solucion[i]] = (1-eps)*tau[mejor_solucion[i+1]][mejor_solucion[i]]+eps*coste(g, mejor_solucion)
    
        iteracion+=1   
    return mejor_solucion, coste(g, mejor_solucion)
    

In [38]:
hormigas()

([7, 6, 5, 4, 3, 2, 1], 381.6699617675482)

In [42]:
hormigas(m = 18,actualizacionLocal=True, filename='./data/US120.txt')[1]
##26160.416487

26986.262407309623

In [38]:
g = Localizaciones(filename='./data/grafo8cidades.txt')
g.nciudades

8

In [8]:
hormigas(filename='./data/US120.txt', actualizacionLocal=True, m=128)

([], 0.0)

In [8]:
coste(g, greedy(g))

381.6699617675482

❓ **Pregunta 1**. Explica brevemente los detalles relevantes de tu código para entender tu implementación (p.ej., estructura de tu código, funciones, implementación de operadores, etc.). Explica también cómo has realizado la verificación de tu implementación.

## P4.2: Laboratorio

Realiza los laboratorios planteados a continuación para estudiar los efectos de los parámetros de la metaheurística. Para esta práctica, reporta también el coste temporal promedio de resolución.

❓ **Pregunta 2**. Realiza el estudio de la calidad de la solución variando el tamaño de la colonia: 1, 2, 4, 8, 16, 32, 64, 128,... ¿Qué valor recomendarías para el problema de las 120 ciudades? ¿Por qué? 

❓ **Pregunta 3**. Ahora realiza un estudio similar variando la probabilidad de explotación q<sub>0</sub> en el rango 0, ..., 0.95 en pasos de 0.05~0.10. ¿Qué valor recomendarías para este parámetro? ¿Por qué?

❓ **Pregunta 4**. Estudia la influencia de pistas de feromonas e información heurística: α=1; β=2, ..., 5. ¿Qué valor de α y β recomendarías? ¿Por qué?

❓ **Pregunta 5**. Finalmente, argumenta de manera crítica las diferencias y similitudes con la metaheurísticas vistas hasta el momento en las interactivas/prácticas anteriores. Discute críticamente acerca de las bondades y limitaciones de las técnicas. Por ejemplo: ¿Eres capaz de resolver instancias de diferente tamaño? ¿Con cuál estás obteniendo mejores soluciones? ¿Cuál parece tener más limitaciones en términos de tiempo de ejecución necesario? ¿Te quedarías con una de las Metaheurísticas en vista de los resultados obtenidos? ¿Cuál/es y por qué?

***Apoya todas tus respuestas en datos-gráficos resultantes de tus estudios***. Para la pregunta 5 no es necesario volver a tomar datos de otras prácticas, pues puedes tomar los resultados y evidencias obtenidas en prácticas anteriores.
No olvides indicar qué valores has establecido para los parámetros relacionados a la condición de parada.

*Importante*: además de la calidad de la soluciones obtenidas, en esta práctica debes medir tiempos para tomar medidas operativas sobre el número de repeticiones que permitan realizar promedios y prescindir de manera razonada de valores en las series de ejecución que no sean computacionalmente rentables/viables con tu implementación/ordenador (p.ej., tamaños de colonia elevados pueden tomar mucho tiempo para resolver).


In [45]:
# puedes añadir aquí la implementación o cualquier código que necesites para responder las preguntas
# utiliza tantas celdas como necesites, pero recuerda que el informe final que resuma las cuestiones 
# no debe superar las 1200 palabras.

import time

#PREGUNTA 2: VARIACION TAMAÑO DE LA COLONIA
tamanhos=[1, 2, 4, 8, 16, 32, 64, 128]
soluciones=[]
tiempos=[]
for i in tamanhos:
    ti=time.time()
    soluciones.append(hormigas(m = i, filename='./data/US120.txt')[1])
    tf=time.time()
    tiempos.append(tf-ti)

solucionesLocal=[]
tiemposLocal=[]
for i in tamanhos:
    ti=time.time()
    solucionesLocal.append(hormigas(actualizacionLocal=True, m = i, filename='./data/US120.txt')[1])
    tf=time.time()
    tiemposLocal.append(tf-ti)

In [44]:
print(soluciones)
print(tiempos)

[27033.84988200304, 26934.56426192349, 26930.325402423787, 26895.485245380507, 26930.325402423787, 26505.783228587337, 26505.783228587337, 25908.79148636877]
[0.19681119918823242, 0.16693520545959473, 0.2612030506134033, 0.4465782642364502, 0.8316423892974854, 1.5795512199401855, 3.1815688610076904, 6.311371088027954]


In [46]:
print(solucionesLocal)
print(tiemposLocal)

[27033.84988200304, 26995.74086032287, 27033.84988200304, 27033.84988200304, 26986.262407309623, 26986.262407309623, 26986.262407309623, 26995.74086032287]
[0.14069604873657227, 0.18529939651489258, 0.3471488952636719, 0.7430050373077393, 1.1295208930969238, 2.374065637588501, 5.971258640289307, 12.137480020523071]


Con actualización local el tiempo aumenta sin proporcionar mejoras notables en las soluciones, por lo que la descartaremos

In [52]:
#PREGUNTA 3: VARIACION DE LA POSIBILIDAD DE EXPLOTACIÓN Q0
soluciones=[]
tiempos=[]
for i in range(1, 20, 1):
    p = i*0.05
    ti=time.time()
    soluciones.append(hormigas(q0=p, m=32, filename='./data/US120.txt')[1])
    tf=time.time()
    tiempos.append(tf-ti)

In [53]:
print(soluciones)
print(tiempos)

[25529.86199688543, 25281.73734869848, 26021.249404413087, 25689.95021662458, 25865.059540151316, 25370.491876632255, 26264.379206377118, 26505.783228587337, 26276.277007668592, 26786.31427083548, 26505.783228587337, 25919.497392097106, 26276.277007668592, 26786.31427083548, 26895.485245380507, 26505.783228587337, 27033.84988200304, 27033.84988200304, 27033.84988200304]
[2.1238672733306885, 2.5008583068847656, 2.2197797298431396, 2.010073184967041, 2.0552291870117188, 1.9045131206512451, 1.7186641693115234, 1.9759948253631592, 2.338858127593994, 1.7447755336761475, 1.5844275951385498, 1.5951793193817139, 1.6723556518554688, 1.5785131454467773, 1.5761363506317139, 1.6566002368927002, 1.6537528038024902, 1.6663010120391846, 1.6984055042266846]


In [60]:
#PREGUNTA 4: INFLUENCIA PISTA DE FEROMONAS E INFORMACIÓN HEURÍSTICA
#variar alfa y beta
soluciones=[]
tiempos=[]
for alfa in range(1, 6):
    for beta in range(2, 6):
        ti=time.time()
        soluciones.append(hormigas(q0=0.30, alfa=alfa, beta=beta, m=32, filename='./data/US120.txt')[1])
        tf=time.time()
        tiempos.append(tf-ti)

1 2
1 3
1 4
1 5
2 2
2 3
2 4
2 5
3 2
3 3
3 4
3 5
4 2
4 3
4 4
4 5
5 2
5 3
5 4
5 5


In [63]:
print(soluciones)
print(tiempos)

[26505.783228587337, 25425.513902770726, 25480.085481295795, 25237.28951995445, 26895.485245380507, 26505.783228587337, 26668.71054550277, 25264.05972196128, 26505.783228587337, 26673.545860945724, 26505.783228587337, 26505.783228587337, 26895.485245380507, 26693.851063386297, 26673.545860945724, 26673.545860945724, 26505.783228587337, 26673.545860945724, 26673.545860945724, 26673.545860945724]
[1.7085762023925781, 1.9309298992156982, 1.824542760848999, 2.014535903930664, 1.8489558696746826, 2.067798614501953, 2.2483623027801514, 2.2836861610412598, 1.8996660709381104, 2.3269660472869873, 2.3645684719085693, 2.419057846069336, 1.6032814979553223, 2.209484577178955, 2.340958833694458, 2.3645541667938232, 1.7790865898132324, 2.9240777492523193, 2.9937171936035156, 3.027139186859131]



#### Respuestas y evaluación

Recordatorio: No olvides escribir tu nombre y apellidos en la segunda celda de este documento.
La respuestas a las preguntas deben venir acompañadas de las implementaciones necesarias para su respuesta.

*P4.1: Implementación básica* (5 puntos)

La implementación básica se evaluará mediante un cuestionario automático de evaluación. Este lo realizarás en la primera sesión tras la entrega, y se centrará en la resolución por tu parte de diversas cuestiones prácticas relacionadas con la implementación realizada, pudiendo ser necesaria la ejecución, adaptación y modificación de la misma.

Pregunta 1. 

Independientemente del cuestionario automático de evaluación, considera siempre que las preguntas planteadas en el notebook deben ser respondidas también. Esas preguntas generales están diseñadas para formarte, y te servián para razonar y reflexionar sobre el tema, así como también para fomentar una discusión constructiva con los docentes en caso de dudas.

*P4.2: Laboratorio* (5 puntos)

El informe a elaborar no debe exceder la longitud máxima de 1200 palabras.

Aclaraciones: La evaluación de esta parte se llevará a cabo en términos de la completitud y correctitud del laboratorio implementado, así como de la calidad del propio informe, que debe ser conciso y preciso, pudiendo acompañarse de gráficas y tablas que faciliten y fundamenten la explicación e argumentación. Es muy importante explicar de manera clara, precisa y fundamentada. Se reservará hasta un punto que se asignará en términos de la calidad de la mejor solución obtenida entre el conjunto de las prácticas entregadas (es por ello que no debes olvidar marcar en tu informe muy claramente cuál ha sido tu mejor solución y con qué configuración/versión).

En este informe se estudiarán diferentes parámetros que afectan al comportamiento del algoritmo de búsqueda basada en poblaciones: colonia de hormigas.

**Pregunta 2** Variación del tamaño de la colonia

En primer lugar, uno de los parámetros que más puede influir es el tamaño de la colonia de hormigas con la que estemos trabajando. Para evaluar esto utilizaremos los tamaños de colonia: [1, 2, 4, 8, 16, 32, 64, 128]. Para estos tamaños se ha guardado el mejor coste obtenido, así como el tiempo que tarda el algoritmo en ejecutarse. Además, se ha realizado el experimento con y sin actualización local.

Con esto, hemos obtenido los siguientes resultados:

<img src='./fotos/tamanhocolonia.png'>

Aquí podemos ver en primer lugar que tanto los costes como los tiempos necesitados en el caso de hacer actualización local son más altos, por lo que a partir de este punto del laboratorio, no la haremos.

En cuanto al tamaño de la colonia, cuando este aumenta, evidentemente también lo hace el tiempo de computación, y a la vez disminuye el coste de la mejor solución. Un mayor número de hormigas equivale a una mayor explroación del espacio de soluciones y por tanto, una mayor probabilidad de encontrar una solución mejor. Sin embargo, también hay que tener en cuenta el tiempo que tardará. 

Para el problema de las 120 ciudades, con un tamaño de población de 32, con el que obtenemos una solución relativaente buena (en este caso de coste 26505.78323) en menos de 2 segundos.


**Pregunta 3** Variación de la probabildad de explotación q0

A continuación realizaremos un experimento para determinar el mejor valor de q0, la probabilidad de explotación. Para este experimento no utilizaremos actualización local, y usaremos un tamaño de población 32, como hemos decidido en el anterior apartado. Con esto, variaremos el valor de q0 entre 0.05 y 0.95. Los resultados obtenidos son los siguientes.

<img src='./fotos/varq0.png'>

En esta gráfica podemos ver en azul los costes obtenidos y en rojo los tiempos necesitados. Vemos que hay bastantes picos, lo que se debe a que, al fin y al cabo, q0 se trata de una probabilidad, por lo que no determina completamente el comportamiento de la ejecución. En general, al aumentar la probabilidad de explotación, los tiempos de computación se disminuyen, mientras que los costes aumentan. Esto sucede porque el comportamiento del algoritmo se hace cada vez menos exploratorio, no diversifica las soluciones exploradas, lo que ocasiona que prácticamente sólo se exploten las primeras soluciones encontradas, dificultando encontrar soluciones diferentes y posiblemente mejores. 

Para encontrar un equilibrio entre diversificación y explotación sería bueno quedarnos con un valor intermedio para q0. En este caso seleccionaremos 0.30, caso para el que obtenemos un coste de 25370.49188 en 1.91 segundos.


**Pregunta 4** Influencia de las pistas de feromonas e información heurística

En este último punto del estudio acerca del algoritmo de colonias de hormigas trataremos la influencia de las pistas de feromonas e información heurística variando los valores de alfa y beta entre 1 y 5. Utilizaremos también los valores para los otros parámetros escogidos hasta ahora. Con todo esto, los resultados obtenidos los podemos ver reflejados en las siguientes gráficas, en las que podemos ver los costes en la primera y los tiempos en la segunda. Para ambas, los menores valores los vemos del tono de azul más oscuro.

<img src='./fotos/alfa_beta_coste.png'>
<img src='./fotos/alfa_beta_tiempos.png'>

Podemos apreciar que la mejor solución en cuanto a coste la obtenemos para alfa=1 y beta=5, aunque las siguientes mejores se encuentran cerca, con beta entre 3 y 4, y alfa entre 1 y 2. En cuanto a tiempo, sin embargo encontramos más variación, sin embargo siempre es más bajo en las combinaciones en las que alguno de los dos valores es muy bajo. 

Para establecer un equilibrio con el que encontrar una buena solución en poco tiempo, nos quedaremos con alfa=1 y beta=5, con lo que conseguimos un tiempo necesario bajo, de **1.786 segundos**, con el que obtenemos una solución con coste final de **25164.79671km**.


**Pregunta 5**

En esta última pregunta compararemos las diferentes metaheurísticas trabajadas durante la asignatura. En la siguiente tabla encontramos los resultados obtenidos para las metaheurísticas estudiadas para su posterior comparación.

Finalmente, argumenta de manera crítica las diferencias y similitudes con la metaheurísticas vistas hasta el momento en las interactivas/prácticas anteriores. Discute críticamente acerca de las bondades y limitaciones de las técnicas. Por ejemplo: ¿Eres capaz de resolver instancias de diferente tamaño? ¿Con cuál estás obteniendo mejores soluciones? ¿Cuál parece tener más limitaciones en términos de tiempo de ejecución necesario? ¿Te quedarías con una de las Metaheurísticas en vista de los resultados obtenidos? ¿Cuál/es y por qué?


In [2]:
from tabulate import tabulate

encabezados=['Metaheurística', 'Mejor coste', 'Tiempo (Segundos)']
datos=[['Temple Simulado', 145646.1618, 0.12],
      ['Búsqueda Tabú', 23806.7467, 295.71],
      ['Algoritmo genético', 49783.3841, 7.24],
      ['Colonias de hormigas', 25164.7967, 1.78]]
tabla = tabulate(datos, headers=encabezados, tablefmt="pretty")
print(tabla)

+----------------------+-------------+-------------------+
|    Metaheurística    | Mejor coste | Tiempo (Segundos) |
+----------------------+-------------+-------------------+
|   Temple Simulado    | 145646.1618 |       0.12        |
|    Búsqueda Tabú     | 23806.7467  |      295.71       |
|  Algoritmo genético  | 49783.3841  |       7.24        |
| Colonias de hormigas | 25164.7967  |       1.78        |
+----------------------+-------------+-------------------+


En primer lugar, podemos ver que el temple simulado es con diferencia el algoritmo para el que peores resultados hemos obtenido. Aunque fue el más rápido de los estudiados, no compensa la diferencia de coste.

El siguiente, la búsqueda tabú, es la que nos aporta la mejor solución de las obtenidas, seguida por las colonias de hormigas con una pequeña diferencia de coste. Sin embargo, la diferencia de tiempo si que es muchísimo más notable. Mientras que para el coste obtenemos un empeoramiento del 5%, el tiempo mejora un 99.40%, por lo que es mucho más eficiente, sobre todo a la hora de trabajar con problemas más grandes, puesto que la búsqueda tabú se haría insostenible. 

En cuanto al algoritmo genético, consigue una solución buena, aunque bastante lejana de las mejores. Sin embargo, lo hace en poco tiempo. En comparación con la búsqueda tabú, consigue una solución un 108.51% peor, es decir, aproximadamente el doble de coste, mientras que de tiempo es un 97.55% más rápido, es decir, obtiene una solución con el doble de coste en un 2.5% del tiempo que necesitaba el algoritmo de búsqueda tabú. Aunque no es el algoritmo de mejores resultados, desde luego no es malo.

Tras comparar estas diferentes metaheurísticas, definitivamente la de colonias de hormigas es la más eficiente puesto que se acerca mucho a la mejor solución posible en un tiempo excelente.
